In [9]:
import cv2
import numpy as np
import requests

from pathlib import Path
from tqdm.notebook import tqdm

In [6]:
image_urls = {
    'img_01.jpg': 'https://www.allplan.com/fileadmin/_processed_/1/c/csm_Petronas-Towers_Wikipedia_20170711_d1c94137d1.jpg',
    'img_02.jpg': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS1uHo9GsI4ZieuGJMOEOK8wJkyKIu6bpMyoQ&s',
    'img_03.jpg': 'https://www.landmarksociety.org/wp-content/uploads/2018/10/general1.jpg',
    'img_04.jpg': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmXR_gtALYWusO2TGAjfrBzDIVI17c2DmnToTHMQKUK6wwtcQXMQGsKobLZNwkM6bMekk&usqp=CAU',
    'img_05.jpg': 'https://commonedge.org/wp-content/uploads/2016/05/New_York_City_building-1-1000x625.jpg',
    'img_06.jpg': 'https://cdn2.vox-cdn.com/thumbor/w0Wj9NbHa4rgTNR5ZbkRkGbFE1U=/640x0/filters:no_upscale()/cdn.vox-cdn.com/uploads/chorus_asset/file/8575085/432park2016.0.png',
    'img_07.jpg': 'https://localyse.eu/wp-content/uploads/2023/07/empty-street-ancient-building-facades-europe-2021-08-30-02-27-54-utc-scaled.jpg',
    'img_08.jpg': 'https://www.pbs.org/wgbh/nova/media/original_images/iStock-1086864102.jpg',
    'img_09.jpg': 'https://digitalspyuk.cdnds.net/16/29/1468833925-oscorp-building-street-view.jpg',
    'img_10.jpg': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSARjfJfWcfZAf1S2rad9KtYX6IrNjxP8vAQA6MUIktYXxQ7_7ngTObqZ8KBPH6t9Q1uGA&usqp=CAU',
    'img_11.jpg': 'https://media.istockphoto.com/id/908031820/photo/lower-manhattan-cityscape-chinatown.jpg?s=612x612&w=0&k=20&c=JBOJ5YIMfJ8bxui1StZcoMSNcysNu8BRbnOSqajpkdM=',
    'img_12.jpg': 'https://variety.com/wp-content/uploads/2018/12/Google-315-Hudson-Street.jpg',
    'img_13.jpg': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/9d/74/9f/photo0jpg.jpg?w=500&h=500&s=1',
    'img_14.jpg': 'https://unimovers.com/blog/wp-content/uploads/sites/9/2024/08/36182055102b4f818cc78b1c44709dd9_716x444.jpg',
    'img_15.jpg': 'https://www.castlecookemortgage.com/hs-fs/hubfs/BG_HomeTypes_Ranch.jpg?width=804&name=BG_HomeTypes_Ranch.jpg',
    'img_16.jpg': 'https://www.zillowstatic.com/bedrock/app/uploads/sites/47/GA_ATL_DOWNTOWN_82792_078-RT_RT_cropRT-5f9e47-1440x960.jpg',
}

In [8]:
base_path = Path('./img/buildings')
#base_path = Path('./img/buildings-finetune')
base_path.mkdir(parents=True, exist_ok=True)

In [9]:
image_paths = []
for img_name, url in tqdm(image_urls.items()):
    response = requests.get(url)
    response.raise_for_status()

    img_data = response.content
    img_path = base_path / img_name
    with open(img_path, 'wb') as f:
        f.write(img_data)
    image_paths.append(img_path)

  0%|          | 0/16 [00:00<?, ?it/s]

In [12]:
import scipy.io as sio
import numpy as np
import os


def build_negative_samples_mat(image_paths, output_mat_file, window_size=32, stride=12):
    if os.path.exists(output_mat_file):
        mat_data = sio.loadmat(output_mat_file)
        existing_X = mat_data['X']  # Shape: (32, 32, 3, n_samples)
        existing_y = mat_data['y']  # Shape: (n_samples, 1)
        n_existing = existing_X.shape[3]
        print(f"Found existing .mat with {n_existing} samples")
    else:
        existing_X = np.zeros((32, 32, 3, 0))
        existing_y = np.zeros((0, 1))
        n_existing = 0
        print("Creating new .mat file")
    
    # Extract patches from images.
    new_patches = []
    for img_path in image_paths:
        print(f'extracting patches from {img_path}')
        img = cv2.imread(img_path)
            
        # Convert BGR to RGB.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        height, width = img.shape[:2]
        patch_count = 0
        for y in range(0, height - window_size, stride):
            for x in range(0, width - window_size, stride):
                patch = img[y:y+window_size, x:x+window_size]
                new_patches.append(patch)
                patch_count += 1
                
        print(f"Extracted {patch_count} patches from {img_path}")
    
    # Convert to numpy array.
    new_patches = np.array(new_patches)
    n_new = len(new_patches)
    
    # Transpose to match SVHN format: height(1), width(2), channels(3), n_samples(0).
    new_patches = np.transpose(new_patches, (1, 2, 3, 0))
    
    # Create labels for new patches.
    new_labels = np.ones((n_new, 1))
    
    # Combine existing and new data.
    combined_X = np.concatenate([existing_X, new_patches], axis=3)
    combined_y = np.concatenate([existing_y, new_labels], axis=0)
    
    # Create updated dictionary.
    updated_dict = {
        'X': combined_X,
        'y': combined_y
    }
    
    # Save to .mat file.
    sio.savemat(output_mat_file, updated_dict, format='5')
    print(f"Added {n_new:_} patches. Total: {n_existing + n_new:_} patches")
    
    return n_existing + n_new

In [12]:
negative_mat_file = 'negative_samples.mat'
build_negative_samples_mat(image_paths, negative_mat_file)

Creating new .mat file
extracting patches from img/buildings/img_01.jpg
Extracted 7236 patches from img/buildings/img_01.jpg
extracting patches from img/buildings/img_02.jpg
Extracted 260 patches from img/buildings/img_02.jpg
extracting patches from img/buildings/img_03.jpg
Extracted 15088 patches from img/buildings/img_03.jpg
extracting patches from img/buildings/img_04.jpg
Extracted 266 patches from img/buildings/img_04.jpg
extracting patches from img/buildings/img_05.jpg
Extracted 4050 patches from img/buildings/img_05.jpg
extracting patches from img/buildings/img_06.jpg
Extracted 1581 patches from img/buildings/img_06.jpg
extracting patches from img/buildings/img_07.jpg
Extracted 17064 patches from img/buildings/img_07.jpg
extracting patches from img/buildings/img_08.jpg
Extracted 21780 patches from img/buildings/img_08.jpg
extracting patches from img/buildings/img_09.jpg
Extracted 12464 patches from img/buildings/img_09.jpg
extracting patches from img/buildings/img_10.jpg
Extracte

102518

# Second dataset

In [4]:
image_urls = {
    'img-01.jpg': 'https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iLnAPfwDkc0Y/v1/1200x800.jpg',
    'img-02.jpg': 'https://www.nj.com/resizer/v2/https%3A%2F%2Fadvancelocal-adapter-image-uploads.s3.amazonaws.com%2Fimage.nj.com%2Fhome%2Fnjo-media%2Fwidth2048%2Fimg%2Fledgerupdates_impact%2Fphoto%2F2018%2F08%2F09%2Fkushner-properties-seeks-financing-for-666-fifth-avenue-property-67bd54151087c50b.jpg?auth=24c31d67ee292e72dc47cddf8f0758aa0922067a40038a9b787f086611949cbf&width=1280&quality=90',
}

In [5]:
base_path = Path('./img/buildings-finetune')
base_path.mkdir(parents=True, exist_ok=True)

In [6]:
image_paths = []
for img_name, url in tqdm(image_urls.items()):
    response = requests.get(url)
    response.raise_for_status()

    img_data = response.content
    img_path = base_path / img_name
    with open(img_path, 'wb') as f:
        f.write(img_data)
    image_paths.append(img_path)

  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
!rm -r ./negative_samples_finetune.mat

In [14]:
negative_mat_file = 'negative_samples_finetune.mat'
build_negative_samples_mat(image_paths, negative_mat_file, window_size=32, stride=14)

Creating new .mat file
extracting patches from img/buildings-finetune/img-01.jpg
Extracted 4620 patches from img/buildings-finetune/img-01.jpg
extracting patches from img/buildings-finetune/img-02.jpg
Extracted 5310 patches from img/buildings-finetune/img-02.jpg
Added 9_930 patches. Total: 9_930 patches


9930

In [2]:
image_urls = {
    'h1.jpg': 'https://i.etsystatic.com/40106094/r/il/ed369a/4601214666/il_1588xN.4601214666_fnpg.jpg',
    'h2.jpg': 'https://backlitledsign.com/cdn/shop/products/personalized-suite-plaque-stainless-steel-number-sign-metal-apartment-plate-rectangle-bespoke-address-plaque-door-number-slate-816350.jpg?v=1710695028&width=720',
    'h3.jpg': 'https://res.cloudinary.com/emtek/image/fetch/f_auto,q_auto,w_1200,c_fill,d_product-placeholder.jpg,ar_8:5,dpr_auto/https%3A%2F%2Fwww.emtek.com%2Fmedia%2Foriginal_images%2FEmtek_Modern_House_Number_Exterior_Install_US19_709x1024px_300dpi_RGB_High.jpg',
    'h4.jpg': 'https://res.cloudinary.com/brickandbatten/images/f_auto,q_auto/v1641215574/wordpress_assets/house-numbers-on-gate-post/house-numbers-on-gate-post.png?_i=AA',
}

In [3]:
base_path = Path('./img/buildings-samples')
base_path.mkdir(parents=True, exist_ok=True)

image_paths = []
for img_name, url in tqdm(image_urls.items()):
    response = requests.get(url)
    response.raise_for_status()

    img_data = response.content
    img_path = base_path / img_name
    with open(img_path, 'wb') as f:
        f.write(img_data)
    image_paths.append(img_path)

  0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
image = cv2.imread('./img/buildings-samples/h0.jpg')
h, w = image.shape[:2]
center = w/2, h/2

angle = 30 # Rotation angle in degrees
scale = 1.0 # Scaling factor (1.0 means no scaling)
M = cv2.getRotationMatrix2D(center, angle, scale)

rotated = cv2.warpAffine(image, M, (w, h))
cv2.imwrite('./img/buildings-samples/h0-rotated.jpg', rotated)

True

In [15]:
image = cv2.imread('./img/buildings-samples/h4.jpg')
row, col, ch = image.shape

mean = 0
sigma = 100
gauss = np.random.normal(mean, sigma, (row, col, ch)).astype(np.float32)
noisy_image = np.clip(image + gauss, 0, 255).astype(np.uint8)

cv2.imwrite('./img/buildings-samples/h4-noisy.jpg', noisy_image)

True